# Benchmark classical approaches

In [1]:
import os
import sys
import nltk
from tqdm import trange
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
from src.preprocessing.corpus_preprocessor import CorpusPreprocessor
from src.preprocessing.functions import *
from src.preprocessing.consts import EMAIL_REGEX
from src.ds_loaders.xsum import XSumLoader
from src.metrics import BLEU, ROUGE
from src.model import *

[nltk_data] Downloading package punkt to /home/pasha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pasha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pasha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/pasha/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2022-05-26 15:45:11.862237: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


DEBUG: Loaded GloVe embedings from /home/pasha/Documents/ML/ML_project_summarization/src/embedings/../../data/glove/glove.6B.100d.txt


## Load dataset

In [2]:
loader = XSumLoader()
loader.load()

Using custom data configuration default
Reusing dataset xsum (/home/pasha/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
orig_documents, orig_summaries = loader.X_val, loader.y_val

## Preprocess dataset

In [4]:
# preprocessor = CorpusPreprocessor(verbose=False).\
#     add(to_lower()).\
#     add(expand_contractions()).\
#     add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
#     add(remove_by_regex([r"\d"])).\
#     add(remove_symbols(string.punctuation.replace('.', ''))).\
#     add(lemmatize())
#     # add(remove_words(set(stopwords.words('english'))))


In [5]:
# documents = []
# summaries = []
# for idx in trange(len(orig_documents)):
#     doc = orig_documents[idx]
#     summ = orig_summaries[idx]
#     if len(doc) == 0 or len(summ) == 0:
#         continue
#     documents.append(preprocessor.transform(doc))
#     summaries.append(preprocessor.transform(summ))

## Create models and metrics and preprocessors

In [21]:
# 1. models
summary_size = 1
TextRank = TextRankModel(summary_size)
LexRank = LexRankModel(summary_size)
Lsa = LsaModel(summary_size)
Luhn = LuhnModel(summary_size)
KL = KLModel(summary_size)
MyTextRank = MyTextRankModel(summary_size)

models = [ TextRank, LexRank, Lsa, Luhn, KL, MyTextRank ]

DEBUG: Loaded GloVe embedings from /home/pasha/Documents/ML/ML_project_summarization/src/embedings/../../data/glove/glove.6B.100d.txt


In [22]:
# 2. metrics
bleu1 = BLEU(1)
bleu2 = BLEU(2)
bleu3 = BLEU(3)
bleu4 = BLEU(4)

rouge1_f1 = ROUGE("rouge1", "fmeasure")
rouge2_f1 = ROUGE("rouge2", "fmeasure")
rougeL_f1 = ROUGE("rougeL", "fmeasure")
rouge1_r = ROUGE("rouge1", "recall")
rouge2_r = ROUGE("rouge2", "recall")
rougeL_r = ROUGE("rougeL", "recall")

metrics = [ bleu1, bleu2, bleu3, bleu4,
            rouge1_f1, rouge2_f1, rougeL_f1, rouge1_r, rouge2_r, rougeL_r ]

In [23]:
# 3. preprocessors
preprocessor_strong = CorpusPreprocessor(name="strong").\
    add(to_lower()).\
    add(expand_contractions()).\
    add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
    add(remove_by_regex([r"\d"])).\
    add(remove_symbols(string.punctuation.replace('.', ''))).\
    add(lemmatize()).\
    add(remove_words(set(stopwords.words('english'))))

preprocessor_mid = CorpusPreprocessor(name="mid").\
    add(to_lower()).\
    add(expand_contractions()).\
    add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
    add(remove_by_regex([r"\d"])).\
    add(remove_symbols(string.punctuation.replace('.', '')))

preprocessor_simple = CorpusPreprocessor(name="simple").\
    add(to_lower()).\
    add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
    add(remove_symbols(string.punctuation.replace('.', '')))

preprocessor_no = CorpusPreprocessor(name="nothing")
preprocessors = [preprocessor_strong, preprocessor_mid, preprocessor_simple, preprocessor_no]

In [24]:
result = {}
def evaluate_models(preprocessors,
                    models,
                    metrics,
                    documents,
                    summaries):
    n_jobs = len(models) * len(preprocessors)
    n_done = 0
    for prep in preprocessors:
        result[str(prep)] = {}
        print(f"Starting with preprocessor {prep}")
        prep_docs = prep.transform(documents)
        prep_sum = prep.transform(summaries)
        for model in models:
            result[str(prep)][str(model)] = {}
            print(f"    Model: {model}")
            model.fit(prep_docs)
            pred = model.predict(prep_docs)
            # print(f"        ", end="")
            for m_idx, metric in enumerate(metrics):
                print(f"      {metric}... ", end="")
                metric.reset_state()
                metric.update_state(pred, prep_sum)
                value = metric.result()
                result[str(prep)][str(model)][str(metric)] = value
                print(f"{value} - {m_idx+1} / {len(metrics)}")
            print()
            n_done += 1
            print(f"Done: {n_done}/{n_jobs}")
    return result

In [26]:
models = [ TextRank, LexRank, Lsa, Luhn, KL]
evaluate_models(
    preprocessors,
    models,
    metrics,
    orig_documents,
    orig_summaries
)

Starting with preprocessor CourpusPreprocessor(strong)
    Model: TextRank


100%|██████████| 11332/11332 [01:11<00:00, 159.57it/s]


      BLEU-1... 0.09680720881076871 - 1 / 10
      BLEU-2... 0.03845039955288756 - 2 / 10
      BLEU-3... 0.015479914618198075 - 3 / 10
      BLEU-4... 0.006538746856412794 - 4 / 10
      ROUGE-1-fmeasure... 0.13007354789886896 - 5 / 10
      ROUGE-2-fmeasure... 0.0208842889198962 - 6 / 10
      ROUGE-L-fmeasure... 0.10526554573311184 - 7 / 10
      ROUGE-1-recall... 0.16140673311994297 - 8 / 10
      ROUGE-2-recall... 0.02659906348806116 - 9 / 10
      ROUGE-L-recall... 0.1306985926412772 - 10 / 10

Done: 1/20
    Model: LexRank


100%|██████████| 11332/11332 [01:15<00:00, 149.35it/s]


      BLEU-1... 0.10481592471646123 - 1 / 10
      BLEU-2... 0.039742980396033 - 2 / 10
      BLEU-3... 0.015753875527716815 - 3 / 10
      BLEU-4... 0.0065695067986235355 - 4 / 10
      ROUGE-1-fmeasure... 0.11485304673723976 - 5 / 10
      ROUGE-2-fmeasure... 0.01652320625328094 - 6 / 10
      ROUGE-L-fmeasure... 0.09676674416946662 - 7 / 10
      ROUGE-1-recall... 0.1174883190936935 - 8 / 10
      ROUGE-2-recall... 0.016932073694746697 - 9 / 10
      ROUGE-L-recall... 0.09850349639780925 - 10 / 10

Done: 2/20
    Model: Lsa


...

100%|██████████| 11332/11332 [00:43<00:00, 259.44it/s]


      BLEU-1... 0.12112938852227359 - 1 / 10
      BLEU-2... 0.042327717238092644 - 2 / 10
      BLEU-3... 0.017866493364940974 - 3 / 10
      BLEU-4... 0.008896621397643781 - 4 / 10
      ROUGE-1-fmeasure... 0.17573443884667378 - 5 / 10
      ROUGE-2-fmeasure... 0.024960913672791008 - 6 / 10
      ROUGE-L-fmeasure... 0.12622903652363654 - 7 / 10
      ROUGE-1-recall... 0.22102281050076938 - 8 / 10
      ROUGE-2-recall... 0.031862008359402325 - 9 / 10
      ROUGE-L-recall... 0.15876822221719286 - 10 / 10

Done: 19/20
    Model: KL


100%|██████████| 11332/11332 [05:42<00:00, 33.06it/s] 


      BLEU-1... 0.1298369096999311 - 1 / 10
      BLEU-2... 0.046572576360524295 - 2 / 10
      BLEU-3... 0.019946079180631883 - 3 / 10
      BLEU-4... 0.009792576427229386 - 4 / 10
      ROUGE-1-fmeasure... 0.17444191867975511 - 5 / 10
      ROUGE-2-fmeasure... 0.02601001773707392 - 6 / 10
      ROUGE-L-fmeasure... 0.13029782298150355 - 7 / 10
      ROUGE-1-recall... 0.197104334621429 - 8 / 10
      ROUGE-2-recall... 0.02948479834138394 - 9 / 10
      ROUGE-L-recall... 0.1474663058305305 - 10 / 10

Done: 20/20


{'CourpusPreprocessor(strong)': {'TextRank': {'BLEU-1': 0.09680720881076871,
   'BLEU-2': 0.03845039955288756,
   'BLEU-3': 0.015479914618198075,
   'BLEU-4': 0.006538746856412794,
   'ROUGE-1-fmeasure': 0.13007354789886896,
   'ROUGE-2-fmeasure': 0.0208842889198962,
   'ROUGE-L-fmeasure': 0.10526554573311184,
   'ROUGE-1-recall': 0.16140673311994297,
   'ROUGE-2-recall': 0.02659906348806116,
   'ROUGE-L-recall': 0.1306985926412772},
  'LexRank': {'BLEU-1': 0.10481592471646123,
   'BLEU-2': 0.039742980396033,
   'BLEU-3': 0.015753875527716815,
   'BLEU-4': 0.0065695067986235355,
   'ROUGE-1-fmeasure': 0.11485304673723976,
   'ROUGE-2-fmeasure': 0.01652320625328094,
   'ROUGE-L-fmeasure': 0.09676674416946662,
   'ROUGE-1-recall': 0.1174883190936935,
   'ROUGE-2-recall': 0.016932073694746697,
   'ROUGE-L-recall': 0.09850349639780925},
  'Lsa': {'BLEU-1': 0.0803511984021305,
   'BLEU-2': 0.028651601505534207,
   'BLEU-3': 0.010071892381747327,
   'BLEU-4': 0.0037947476650204777,
   'ROUGE

In [28]:
from pprint import pprint
import json
pprint(result)
with open("bench_results.json", 'w') as f:
    json.dump(result, f, indent=4)

{'CourpusPreprocessor(mid)': {'KL': {'BLEU-1': 0.14889495406736786,
                                     'BLEU-2': 0.05524890100262196,
                                     'BLEU-3': 0.024375345550722648,
                                     'BLEU-4': 0.012407283399724722,
                                     'ROUGE-1-fmeasure': 0.1732041720417614,
                                     'ROUGE-1-recall': 0.19718918428801246,
                                     'ROUGE-2-fmeasure': 0.02486516104461113,
                                     'ROUGE-2-recall': 0.028462736257747704,
                                     'ROUGE-L-fmeasure': 0.1304430548450467,
                                     'ROUGE-L-recall': 0.14876444060374563},
                              'LexRank': {'BLEU-1': 0.15236908696095924,
                                          'BLEU-2': 0.05438131186722801,
                                          'BLEU-3': 0.023584992852121775,
                                          'B

In [13]:
# small_ds = [orig_documents[0], orig_documents[1]]
# small_ds_prep = preprocessor_simple.transform(small_ds)
# # print(small_ds_prep)
#
# # bads = string.punctuation.replace('.', '')
# # print(bads)
# #
# # tmp = remove_symbols(bads)(orig_documents[1])
# MyTextRank.fit(small_ds_prep)
# res = MyTextRank.predict(small_ds_prep)
# # print(tmp)
# # print("=" * 30)
# # print(small_ds_prep[1])
# # print("=" * 30)
# print(len(orig_documents[1].split(".")), len(nltk.sent_tokenize(orig_documents[1])))
# print("=" * 30)
# print(res[1])

100%|██████████| 2/2 [00:00<00:00, 103.39it/s]

12 8
the 37yearold has scored 230 runs in four firstclass games this season at an average of 57.50.
losing adam is naturally a blow as he contributes significantly to everything we do director of cricket angus fraser said.
